In [ ]:
#playing with https://www.bambooweekly.com/p/bw-18-world-population

In [1]:
import pandas as pd
import numpy as np
from buckaroo.buckaroo_widget import BuckarooWidget, disable, enable

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [ ]:
!cp /Users/paddy/Downloads/WPP2022_Demographic_Indicators_Medium.zip ./

In [ ]:
!unzip WPP2022_Demographic_Indicators_Medium.zip

In [ ]:
def get_outlier_idxs(ser):
    outlier_idxs = []
    try:
        idxs = ser.sort_values().index
    except Exception as e:
        print(e)
        idxs = ser.index
    outlier_idxs.extend(idxs[:5])
    outlier_idxs.extend(idxs[-5:])
    return outlier_idxs

def sample(df, sample_size=500, include_outliers=True):
    if len(df) <= sample_size:
        return df
    sdf = df.sample(np.min([sample_size, len(df)]))
    if include_outliers:
        outlier_idxs = []
        for col in df.columns:
            outlier_idxs.extend(get_outlier_idxs(df[col]) )
        outlier_idxs.extend(sdf.index)
        uniq_idx = np.unique(outlier_idxs)
        return df.iloc[uniq_idx]
    return sdf


In [2]:
df = pd.read_csv('WPP2022_Demographic_Indicators_Medium.csv')
df

/var/folders/th/jfr8ccjx09g2j_18ty_t3xnh0000gn/T/ipykernel_83712/944981008.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('WPP2022_Demographic_Indicators_Medium.csv')


'<' not supported between instances of 'float' and 'str'
error reordering columns cannot reindex on an axis with duplicate labels


/Users/paddy/buckaroo/buckaroo/summary_stats.py:148: FutureWarning: reindexing with a non-unique Index is deprecated and will raise in a future version.
  sdf.loc['is_duplicate':, duplicate_cols] = -5


BuckarooWidget(commandConfig={'argspecs': {'dropcol': [None], 'to_datetime': [None], 'safeint': [None], 'filln…

In [ ]:
from buckaroo.summary_stats import reorder_columns

In [ ]:
reorder_columns(df)

In [ ]:
def probable_datetime(ser):
    s_ser = ser.sample(np.min([len(ser), 500]))
    try:
        dt_ser = pd.to_datetime(s_ser)
        #pd.to_datetime(1_00_000_000_000_000_000) == pd.to_datetime('1973-01-01') 
        if dt_ser.max() < pd.to_datetime('1973-01-01'):
            return False
        return True
        
    except Exception as e:
        return False
#probable_datetime(df['start station name'])


def summarize_string(ser):
    l = len(ser)
    val_counts = ser.value_counts()
    distinct_count= len(val_counts)
    nan_count = l - len(ser.dropna())
    unique_count = len(val_counts[val_counts==1])
    empty_count = val_counts.get('', 0)

    #[pd.api.types.is_integer_dtype(df2[col]) for col in df2.columns]
    #[pd.api.types.is_numeric_dtype(df2[col]) for col in df2.columns]

    return dict(
        dtype=ser.dtype,
        length=l,
        nan_count = nan_count,
        distinct_count= distinct_count,
        empty_count = empty_count,
        empty_per = empty_count/l,
        unique_per = unique_count/l,
        nan_per = nan_count/l,
        is_numeric=pd.api.types.is_numeric_dtype(ser),
        is_integer=pd.api.types.is_integer_dtype(ser),
        is_datetime=probable_datetime(ser),
        mode=ser.mode().values[0])

def summarize_numeric(ser):

    num_stats = dict(
        min=ser.min(),
        max=ser.max(),
        mean=ser.mean())
    num_stats.update(summarize_string(ser))
    return num_stats

def summarize_column(ser):
    if pd.api.types.is_numeric_dtype(ser.dtype):
        return summarize_numeric(ser)
    else:
        return summarize_string(ser)

def summarize_df(df):
    summary_df = pd.DataFrame({col:summarize_column(df[col]) for col in df})
    return summary_df



def set_when(df, cond_row_name, target_row_name, true_val, false_val):
    true_row = df.loc[cond_row_name]
    df.loc[target_row_name] = false_val
    df.loc[target_row_name, true_row[true_row==True].index.values] = true_val
    return df


def without(arr, search_keys):
    new_arr = []
    for v in arr:
        if v not in search_keys:
            new_arr.append(v)
    return new_arr


def find_groupings(corr_pairs):
    all_groupings = []
    for key, other_key_list in corr_pairs.items():
        ab = other_key_list.copy()
        ab.append(key)
        all_groupings.append(set(ab))
    return np.unique(all_groupings)

def reorder_columns(df):
    tdf_stats = summarize_df(df)
    cpd = get_cor_pair_dict(df, tdf_stats)
    col_order = order_columns(tdf_stats, cpd)
    return df[col_order]

In [ ]:
def make_num_categorical(ser):
    ser_uniq = ser.dropna().unique()
    name_to_idx = {name:idx for idx, name in enumerate(ser_uniq)}
    #needs to be vectorized
    num_categorical = ser.dropna().apply(lambda x:name_to_idx[x])
    return num_categorical
#make_num_categorical(df['Notes'])

In [ ]:

def get_cor_pair_dict(df, summary_stats):
    #we need to remove columns with only nans or a single value, they mess up corr()

    #this needs to be vectorized
    corrable_cols = [col for col in df if summary_stats[col]['distinct_count'] > 1]
    #print("corrable_cols", corrable_cols)
    #num_df =  pd.DataFrame({col:numerize_column(df[col]) for col in corrable_cols})

    for col in df.columns:
        make_num_categorical(df[col])
    num_df =  pd.DataFrame({col:make_num_categorical(df[col]) for col in corrable_cols})

    corr_df = num_df.corr()
    high_corr_df = corr_df[corr_df > 0.99]
    cor_dict = {}
    for col in high_corr_df.columns:
        #columns with high correlation that aren't the column itself
        other_cor_cols = high_corr_df[col].dropna().drop(col)
        cor_cols = other_cor_cols.index.values
        if len(cor_cols) > 0:
            cor_dict[col] = cor_cols.tolist()
    return cor_dict


def order_groupings(grps, ranked_cols):
    first_cols, rest_cols = [], []
    for col in ranked_cols:
        for grp in grps:
            if col in grp:
                first_cols.append(col)
                rest_cols.extend(list(without(grp, [col])))
                grps = without(grps, [grp])
    return first_cols, rest_cols

def order_columns(summary_stats_df, corr_pair_dict):
    sdf = summary_stats_df.copy()
    sdf.loc['one_distinct'] = 0

    only_ones = (sdf.loc['distinct_count'] <= 1)
    sdf.loc['one_distinct', only_ones[only_ones==True].index.values] = -20
    
    sdf.loc['first_col'] = 0
    sdf.loc['is_duplicate'] = 0
    set_when(sdf, 'is_datetime', 'datetime_score', 11, 0)
    
    set_when(sdf, 'is_integer', 'grouping_score_integer', -3, 0)
    set_when(sdf, 'is_numeric', 'grouping_score_numeric', -3, 5)
    grouping_col_scores = sdf.loc[['grouping_score_integer', 'grouping_score_numeric']].sum()
    duplicate_col_rankings = grouping_col_scores.sort_values().index[::-1].values

    groupings = find_groupings(corr_pair_dict)
    first_cols, duplicate_cols = order_groupings(groupings, duplicate_col_rankings)
    
    sdf.loc['first_col':, first_cols] = 5
    
    print(first_cols)
    print(duplicate_cols)
    sdf.loc['is_duplicate':, duplicate_cols] = -5
    
    col_scores = sdf.loc[['one_distinct', 'first_col', 'datetime_score', 'is_duplicate']].sum()
    return col_scores.sort_values().index.values[::-1]


In [ ]:
sdf = summarize_df(df)
cpd = get_cor_pair_dict(df, sdf)
cpd

In [ ]:
disable()

In [ ]:
sdf

In [ ]:
dups = ['ISO2_code', 'LocID', 'SortOrder', 'Location', 'TPopulation1Jan', 'Q0060Male', 'TPopulation1July', 'Q1560', 'ISO2_code', 'Q0060', 'SDMX_code', 'Location', 'Q0060Female', 'TPopulation1Jan', 'Q0060Male', 'ISO2_code', 'SDMX_code', 'Location', 'TPopulation1Jan', 'Q0060Male', 'TPopulation1July', 'Q1560', 'ISO2_code', 'Q0060', 'SDMX_code', 'Location', 'Q0060Female', 'TPopulation1Jan', 'Q1560Male', 'Q0060Male', 'TPopulation1July', 'Q1560', 'ISO2_code', 'Q0060', 'SDMX_code', 'Location', 'TPopulation1Jan', 'TPopulation1July', 'TPopulationFemale1July', 'ISO2_code', 'SDMX_code', 'Location', 'TPopulation1Jan', 'Q1560Male', 'Q0060Male', 'TPopulation1July', 'ISO2_code', 'SDMX_code', 'Location', 'Q0060Female', 'TPopulation1Jan', 'Q1560Male', 'Q0060Male', 'TPopulation1July', 'Q1560', 'TPopulationMale1July', 'ISO2_code', 'Q0060', 'SDMX_code', 'TPopulationFemale1July', 'Q0060Female', 'Location', 'PopDensity', 'TPopulation1Jan', 'Q1550Male', 'Q1560Male', 'Q0060Male', 'TPopulation1July', 'Q1560', 'TPopulationMale1July', 'ISO2_code', 'Q0060', 'SDMX_code', 'Q1560Female', 'TPopulationFemale1July', 'ISO2_code', 'ParentID', 'Location', 'Q0060Female', 'TPopulation1Jan', 'Q1550Male', 'Q1560Male', 'Q0060Male', 'TPopulation1July', 'Q1560', 'TPopulationMale1July', 'ISO2_code', 'Q0060', 'SDMX_code', 'Q1560Female', 'TPopulationFemale1July', 'ISO2_code', 'LocID', 'SortOrder', 'SortOrder', 'Location', 'Q0060Female', 'TPopulation1Jan', 'Q1560Male', 'ParentID', 'Q0060Male', 'TPopulation1July', 'Q1560', 'TPopulationFemale1July', 'ISO2_code', 'Q0060', 'LocID', 'SDMX_code', 'Location', 'TPopulation1Jan', 'Q1560Male', 'Q0060Male', 'TPopulation1July', 'Q1560', 'TPopulationMale1July', 'ISO2_code', 'Q0060', 'SDMX_code', 'TPopulationFemale1July', 'LocTypeID', 'Q1550Male', 'SDMX_code', 'PopDensity', 'TPopulation1Jan', 'TPopulation1July', 'TPopulationMale1July', 'SDMX_code', 'SDMX_code', 'Q1560Female', 'DoublingTime']

In [ ]:
len(dups),len(np.unique(dups))

In [ ]:
reorder_columns(df)

In [ ]:
#df['Notes']